In [1]:
param_x_api_key = '2BeBXg68o3pSd6GQvdyS5erm1aSyJUdKxZcgwsrEukJHa1mYjko5'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '{"getAutoCarePartDetails":{"brandCode":"' + input_.loc[a, 'Row_Id'].split(']-[')[0].replace('[', '').strip() + '","partNumber":"' + input_.loc[a, 'Row_Id'].split(']-[')[1].replace(']', '').strip() + '"}}'

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                part_number = json_data['piesItem']['partNumber'].strip()
                
                # = = = = = = = = = = = = = = =
                
                brand_id = json_data['piesItem']['brandCode'].strip()
                
                # = = = = = = = = = = = = = = =
                
                brand = json_data['piesItem']['brandName'].strip()
                
                # = = = = = = = = = = = = = = =
                                
                manufacturer_website = json_data['piesItem']['brandAdditionalInfo']['websiteURL'].strip()

                # = = = = = = = = = = = = = = =
                
                gtin_ean = ''
                if 'itemLevelGTIN' in json_data['piesItem']:
                    gtin_ean = json_data['piesItem']['itemLevelGTIN']['gtin'].strip()

                # = = = = = = = = = = = = = = =
                
                type_of_part = json_data['piesItem']['categoryName'].strip() + ' > ' + json_data['piesItem']['subCategoryName'].strip() + ' > ' + json_data['piesItem']['partTypeName'].strip()
                
                # = = = = = = = = = = = = = = =
                
                environmentally_hazardous_material = ''
                if 'hazardousMaterialCode' in json_data['piesItem']:
                    environmentally_hazardous_material = json_data['piesItem']['hazardousMaterialCode']
                
                # = = = = = = = = = = = = = = =
                
                aces_applications_available = json_data['piesItem']['hasACESApplications']

                # = = = = = = = = = = = = = = =
                
                number_of_parts = str(json_data['piesItem']['itemQuantitySize']['value']) + ' ' + json_data['piesItem']['itemQuantitySize']['uomName'].strip()

                # = = = = = = = = = = = = = = =
                
                type_of_container = json_data['piesItem']['containerTypeUOMName'].strip()

                # = = = = = = = = = = = = = = =
                
                quantity_per_application = str(json_data['piesItem']['quantityPerApplication']['value']) + ' ' + json_data['piesItem']['quantityPerApplication']['uomName'].strip()

                # = = = = = = = = = = = = = = =
                
                minimum_order = ''
                if 'minimumOrderQuantity' in json_data['piesItem']:
                    minimum_order = str(json_data['piesItem']['minimumOrderQuantity']['value']) + ' ' + json_data['piesItem']['minimumOrderQuantity']['uomName'].strip()

                # = = = = = = = = = = = = = = =
                
                list_description = json_data['piesItem']['descriptions']
                
                # = = = = = = = = = = = = = = =
                
                list_extended_information = json_data['piesItem']['extendedInformation']

                # = = = = = = = = = = = = = = =
                
                list_product_attribute = json_data['piesItem']['productAttributes']
                
                # = = = = = = = = = = = = = = =
                
                list_package = json_data['piesItem']['packages']
                
                # = = = = = = = = = = = = = = =
                
                oe = ';'.join([oe['partNumber'].strip() for oe in json_data['piesItem']['partInterchanges']])

                # = = = = = = = = = = = = = = =

                src = ''
                list_src = json_data['piesItem']['digitalAssets']
                for src_ in list_src:
                    if 'imageURL3200' in src_:
                        src = src_['imageURL3200'].strip()
                        break
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Brand_Id': brand_id,
                                         'Brand': brand,
                                         'Manufacturer_Website': manufacturer_website,
                                         'GTIN_EAN': gtin_ean,
                                         'Type_Of_Part': type_of_part,
                                         'Environmentally_Hazardous_Material': environmentally_hazardous_material,
                                         'ACES_Applications_Available': aces_applications_available,
                                         'Number_Of_Parts': number_of_parts,
                                         'Type_Of_Container': type_of_container,
                                         'Qantity_Per_Aplication': quantity_per_application,
                                         'Minimum_Order': minimum_order,
                                         'Vehicle': '',
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': 'https://web.tecalliance.net/walkerproducts/qa/parts/' + brand_id + '/' + part_number + '/ap-detail',
                                         'Src': src,
                                         'Vehicle_Page': '',
                                         'JOIN_PNBI': part_number + ';' + brand_id}])

                for description in list_description:
                    df_temp.loc[0, 'Description_' + description['descriptionTypeName'].strip() + ' (' + description['languageCode'].strip() + ')'] = description['value'].strip()

                for extended_information in list_extended_information:
                    df_temp.loc[0, 'Additional_' + extended_information['expiName'].strip()] = extended_information['formattedValue'].strip()

                for product_attribute in list_product_attribute:
                    if 'customAttributeName' in product_attribute:
                        df_temp.loc[0, 'Attribute_' + product_attribute['customAttributeName'].strip()] = product_attribute['value'].strip()
                    else:
                        df_temp.loc[0, 'Attribute_' + product_attribute['padbAttributeName'].strip()] = product_attribute['value'].strip() + ' ' + product_attribute['uomCode'].strip() if 'uomCode' in product_attribute else product_attribute['value'].strip()

                for i in range(len(list_package)):
                    for name in list_package[i]:
                        if name == 'dimensions':
                            df_temp.loc[0, 'Package_' + str(i+1) + '_height'] = str(list_package[i][name]['height']) + ' ' + list_package[i][name]['uomName']
                            df_temp.loc[0, 'Package_' + str(i+1) + '_width'] = str(list_package[i][name]['width']) + ' ' + list_package[i][name]['uomName']
                            df_temp.loc[0, 'Package_' + str(i+1) + '_length'] = str(list_package[i][name]['length']) + ' ' + list_package[i][name]['uomName']
                        elif name == 'weights':
                            df_temp.loc[0, 'Package_' + str(i+1) + '_weight'] = str(list_package[i][name]['weight']) + ' ' + list_package[i][name]['uomName']
                        elif list_package[i][name] == []:
                            pass
                        else:
                            df_temp.loc[0, 'Package_' + str(i+1) + '_' + name.strip()] = list_package[i][name]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Row_Id': input_.loc[a, 'Row_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Row_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1

[ok] - [DKHD]-[210-1146]
[尝试次数：4] - [剩余数量：0] - [当前时间：09:41:46]

输出ing...

Done ~
